# Demo Script for Milestone 10.15

## 0 Prepare task

### 0.1 prepare planning scene

In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src')))

from pkg.global_config import RNB_PLANNING_DIR
from demo_utils.kiro_udp_send import start_mobile_udp_thread, send_pose_wait, get_xyzw_cur, get_reach_state
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from pkg.project_config import *
from demo_utils.streaming import *
from demo_utils.detect_table import *
from demo_utils.area_select import *
from pkg.detector.aruco.marker_config import get_aruco_map
aruco_map = get_aruco_map()


CONNECT_CAM = False # True
ENABLE_DETECT = False
DETECT_MARKER = True
CONNECT_INDY = False # True
CONNECT_MOBILE = False # True 
SHOW_MOTION_RVIZ = False

# Tool dimensions
TOOL_DIM = [0.32, 0.08]
TOOL_OFFSET = -0.04
ROBOT_Z_ANGLE = np.pi
MARGIN = 0
TRACK_THICKNESS = 0.001

INDY_BASE_OFFSET = (0.172,0,0.439)
ROBOT_Z_ANGLE = np.pi
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001

ip_cur =  get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"
CAM_HOST = '192.168.0.10'

print("Current PC IP: {}".format(ip_cur))
print("Mobile ROB IP: {}".format(MOBILE_IP))
print("CAM SERVER IP: {}".format(CAM_HOST))

from pkg.geometry.builder.scene_builder import SceneBuilder
from demo_utils.environment import *

sock_mobile, server_thread = start_mobile_udp_thread(recv_ip=ip_cur)

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                MOBILE_IP)
robot_config = RobotConfig(1, RobotType.indy7, (INDY_BASE_OFFSET, (0,0,np.pi)),
                INDY_IP, root_on="kmb0_platform", specs={"no_sdk":True})
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[False, CONNECT_INDY])

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.set_workspace_boundary(-3, 7, -5, 5, -CLEARANCE, 3, thickness=WALL_THICKNESS)


from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE

viewpoint = add_cam(gscene, tool_link=TIP_LINK)
add_indy_tool_kiro(gscene, tool_link=TIP_LINK, face_name=TOOL_NAME, zoff=TOOL_OFFSET)

HOME_POSE = -crob.home_pose
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

# Set planner
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False, incremental_constraint_motion=True)
mplan.motion_filters = [GraspChecker(pscene)]
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

# Register binders
from pkg.planning.constraint.constraint_actor import VacuumTool, Gripper2Tool, PlacePlane, SweepFramer, WayFramer
brush_face = pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
                                  point=(-gscene.NAME_DICT['brush_face'].dims[0]/2-CLEARANCE,0,0), 
                                  rpy=(0,np.pi/2*1,0))

# waypoint
WP_DIMS = (0.6,0.4,WALL_THICKNESS)
gscene.create_safe(gtype=GEOTYPE.BOX, name="wayframer", link_name=HOLD_LINK,
                   dims=WP_DIMS, center=(0,0,WP_DIMS[2]/2), rpy=(0,0,0), color=(1, 0, 0, 0.5), display=True,
                   collision=False, fixed=True)
wayframer = pscene.create_binder(bname="wayframer", gname="wayframer", _type=WayFramer, 
                                 point=(0,0,-WP_DIMS[2]/2-CLEARANCE), rpy=(0,0,0))

Current PC IP: 192.168.0.123
Mobile ROB IP: 192.168.0.102
CAM SERVER IP: 192.168.0.10
connection command:
kmb0: False
indy1: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


### 0.2 Wait task start queue

## 1. Bed cleaning

### 1.1 Detect bed

#### 1.1.1 Move to bed-seek pose 

In [2]:
VIEW_POSE = np.deg2rad([  0., -50.,  70.,  -0.,  100., -180])
VIEW_LOC = [0,]*6
VIEW_POSE_EXT = np.array(VIEW_LOC + list(VIEW_POSE))
if CONNECT_INDY:
    with indy:
        indy.joint_move_to(np.rad2deg(VIEW_POSE))
        time.sleep(0.5)
        indy.wait_for_move_finish()
        Qcur = np.deg2rad(indy.get_joint_pos())
else:
    Qcur = VIEW_POSE
gscene.show_pose(VIEW_POSE_EXT)

#### 1.1.2  detect bed and add to the scene

In [3]:
bed_center = (2,0,0)
bed_rpy = (0,0,np.pi/2)
COLOR_BED_COL = (0,1,0,0.3)

bed_mat = add_bed(gscene, bed_center, bed_rpy, COLOR_BED_COL)

closet_left, closet_rightup, closet_rightdown = add_closet(
    gscene, closet_center=np.matmul(Rot_rpy(bed_rpy), (-0.75,-1,0))+bed_center, closet_rpy=bed_rpy, 
    COLOR_CLOSET_COL=(0,1,0,0.3))

### 1.2 Make bed cleaning plan

#### 1.2.1 get division-base pose combination data

In [4]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_actor import *
from pkg.planning.constraint.constraint_subject import *

In [5]:
gcheck = GraspChecker(pscene)
mplan.motion_filters = [gcheck]

wp_task, wp_hdl = add_waypoint_task(pscene, "waypoint", WP_DIMS, (0,0,0), (0,0,0), 
                                    parent="floor_ws", color=(0, 0, 1, 0.5))
ccheck = CachedCollisionCheck(gcheck, wp_task, wp_hdl, wayframer)

In [6]:
BED_OFFSET = 0.05
brush_face = pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
                                  point=(-gscene.NAME_DICT['brush_face'].dims[0]/2-CLEARANCE-BED_OFFSET,0,0), 
                                  rpy=(0,np.pi/2*1,0))

T_e_brush = brush_face.get_tf_handle(crob.home_dict, from_link=TIP_LINK)
T_brush_e = SE3_inv(T_e_brush)
EE_HEIGHT = round(bed_mat.get_tf(HOME_DICT)[2,3] + bed_mat.dims[2]/2, 5) + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]

In [7]:
ccheck.clear()
div_base_dict, Tsm_keys, surface_div_centers, sqdiv_size, div_num, (ax_step, ax_swp, ax_pln) = \
                        get_division_dict(bed_mat, brush_face, robot_config, 
                                          plane_val=EE_HEIGHT, tip_dir=None, TOOL_DIM=TOOL_DIM, 
                                          ccheck=ccheck)

#### 1.2.2 select base poses and generate motions

In [8]:
HOME_POSE_SWEEP = VIEW_POSE_EXT
crob.home_pose = HOME_POSE_SWEEP
crob.home_dict = list2dict(crob.home_pose, gscene.joint_names)
floor_ws = gscene.NAME_DICT["floor_ws"]    
test_fun = TestBaseDivFunc(ppline, floor_ws, bed_mat, sqdiv_size, WP_DIMS, TOOL_DIM, crob.home_dict)

In [9]:
test_fun.clear()

idx_bases, idc_divs, covered_all, snode_schedule_dict = select_max_cover_bases(
    div_base_dict, Tsm_keys, surface_div_centers, div_num, ax_step,
    test_fun=test_fun)

Use 36/36 agents
Goal reached
======================= terminated 26: first answer acquired from other agent ===============================
======================= terminated 8: first answer acquired ===============================
======================= terminated 23: first answer acquired from other agent ===============================
======================= terminated 4: first answer acquired from other agent ===============================
======================= terminated 3: first answer acquired from other agent ===============================
Goal reached
======================= terminated 24: first answer acquired from other agent ===============================
======================= terminated 10: first answer acquired ===============================
======================= terminated 2: first answer acquired from other agent ===============================
======================= terminated 9: first answer acquired from other agent ===============================
======

Goal reached
======================= terminated 35: first answer acquired ===============================
======================= terminated 6: first answer acquired from other agent ===============================
======================= terminated 25: first answer acquired from other agent ===============================
======================= terminated 27: first answer acquired from other agent ===============================
======================= terminated 7: first answer acquired from other agent ===============================
======================= terminated 15: first answer acquired from other agent ===============================
Goal reached
======================= terminated 30: first answer acquired ===============================
======================= terminated 33: first answer acquired from other agent ===============================
Goal reached
======================= terminated 14: first answer acquired ===============================
Goal reached
===========

======================= terminated 24: first answer acquired ===============================
======================= terminated 30: first answer acquired from other agent ===============================
Goal reached
======================= terminated 26: first answer acquired ===============================
Goal reached
======================= terminated 4: first answer acquired ===============================
======================= terminated 31: first answer acquired from other agent ===============================
Goal reached
======================= terminated 19: first answer acquired ===============================
======================= terminated 29: first answer acquired from other agent ===============================
======================= terminated 22: first answer acquired from other agent ===============================
Goal reached
======================= terminated 35: first answer acquired ===============================
Goal reached
======================= termina

======================= terminated 4: first answer acquired from other agent ===============================
======================= terminated 24: first answer acquired from other agent ===============================
======================= terminated 18: first answer acquired from other agent ===============================
======================= terminated 29: first answer acquired from other agent ===============================
======================= terminated 32: first answer acquired from other agent ===============================
Goal reached
======================= terminated 2: first answer acquired ===============================
======================= terminated 9: first answer acquired from other agent ===============================
======================= terminated 19: first answer acquired from other agent ===============================
======================= terminated 11: first answer acquired from other agent ===============================
Goal reached
====

======================= terminated 34: first answer acquired from other agent ===============================
Goal reached
======================= terminated 13: first answer acquired ===============================
Goal reached
======================= terminated 26: first answer acquired ===============================
Goal reached
======================= terminated 0: first answer acquired ===============================
Goal reached
======================= terminated 7: first answer acquired ===============================
Use 36/36 agents
Goal reached
======================= terminated 1: first answer acquired ===============================
======================= terminated 3: first answer acquired from other agent ===============================
======================= terminated 26: first answer acquired from other agent ===============================
======================= terminated 34: first answer acquired from other agent ===============================
=================

...

======================= terminated 25: first answer acquired ===============================
Goal reached
======================= terminated 26: first answer acquired ===============================
Goal reached
======================= terminated 17: first answer acquired ===============================
Goal reached
======================= terminated 8: first answer acquired ===============================
Goal reached
Goal reached
======================= terminated 1: first answer acquired ===============================
======================= terminated 15: first answer acquired ===============================
Goal reached
======================= terminated 10: first answer acquired ===============================
Goal reached
======================= terminated 33: first answer acquired ===============================
Goal reached
======================= terminated 4: first answer acquired ===============================
Goal reached
======================= terminated 3: first answe

======================= terminated 22: max iteration time reached (1633464392/1633464389.11 s) ===============================
======================= terminated 20: max iteration time reached (1633464392/1633464389.11 s) ===============================
======================= terminated 10: max iteration time reached (1633464392/1633464389.11 s) ===============================
======================= terminated 24: max iteration time reached (1633464392/1633464389.11 s) ===============================
======================= terminated 18: max iteration time reached (1633464392/1633464389.11 s) ===============================
======================= terminated 27: max iteration time reached (1633464392/1633464389.11 s) ===============================
======================= terminated 5: max iteration time reached (1633464392/1633464389.11 s) ===============================
======================= terminated 2: max iteration time reached (1633464392/1633464389.11 s) ==================

======================= terminated 12: first answer acquired from other agent ===============================
======================= terminated 10: first answer acquired from other agent ===============================
Goal reached
======================= terminated 7: first answer acquired from other agent ===============================
======================= terminated 19: first answer acquired ===============================
Goal reached
======================= terminated 25: first answer acquired ===============================
======================= terminated 33: first answer acquired from other agent ===============================
======================= terminated 1: first answer acquired from other agent ===============================
======================= terminated 15: first answer acquired from other agent ===============================
======================= terminated 26: first answer acquired from other agent ===============================
====================

======================= terminated 9: first answer acquired from other agent ===============================
======================= terminated 21: first answer acquired from other agent ===============================
Goal reached
======================= terminated 18: first answer acquired ===============================
======================= terminated 22: first answer acquired from other agent ===============================
Goal reached
======================= terminated 17: first answer acquired ===============================
Goal reached
======================= terminated 0: first answer acquired ===============================
======================= terminated 29: first answer acquired from other agent ===============================
Use 36/36 agents
Goal reached
Goal reached
======================= terminated 22: first answer acquired from other agent ===============================
Goal reached
======================= terminated 10: first answer acquired =================

Goal reached
======================= terminated 25: first answer acquired ===============================
======================= terminated 34: first answer acquired from other agent ===============================
======================= terminated 12: first answer acquired from other agent ===============================
======================= terminated 15: first answer acquired from other agent ===============================
======================= terminated 16: first answer acquired from other agent ===============================
======================= terminated 20: first answer acquired from other agent ===============================
Goal reached
======================= terminated 9: first answer acquired ===============================
Goal reached
======================= terminated 21: first answer acquired ===============================
Goal reached
======================= terminated 32: first answer acquired ===============================
Goal reached
Goal reached
=

#### 1.2.3 refine motions

In [10]:
HOME_POSE_MOVE = VIEW_POSE
snode_schedule_list, scene_args_list, scene_kwargs_list = refine_order_plan(
    ppline, snode_schedule_dict, idx_bases, idc_divs, Qcur, 
    floor_ws, wayframer, bed_mat, Tsm_keys, surface_div_centers, sqdiv_size, 
    WP_DIMS, TOOL_DIM, ROBOT_NAME, MOBILE_NAME, HOME_POSE_MOVE)

Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached


### 1.3 Execute bed cleaning sequence

In [11]:
# indy.traj_vel = 3

In [12]:
swp_fin_list = []
for i_s, (snode_schedule, sargs, skwargs) in enumerate(zip(snode_schedule_list, scene_args_list, scene_kwargs_list)):
    set_base_sweep(*sargs, **skwargs)
    for snode_pre, snode_nxt in zip(snode_schedule[:-1], snode_schedule[1:]):
        from_state = snode_pre.state
        to_state = snode_nxt.state
        subjects, ok = pscene.get_changing_subjects(from_state, to_state)

        if len(subjects) ==0 or subjects[0] == "sweep":
            if CONNECT_INDY:
                ppline.execute_schedule([snode_pre, snode_nxt], one_by_one=True)
                with indy:
                    time.sleep(0.5)
                    indy.wait_for_move_finish()
            else:
                ppline.play_schedule([snode_pre, snode_nxt])

        elif subjects[0] == "waypoints":
            Tbw0 = wayframer.get_tf_handle(list2dict(from_state.Q, gscene.joint_names))
            Tbw1 = wayframer.get_tf_handle(list2dict(to_state.Q, gscene.joint_names))
            Tw0w1 = np.matmul(SE3_inv(Tbw0), Tbw1)
            cur_xyzw, tar_xyzw_rd, tar_xyzw = get_relative_mobile_command(Tw0w1, CONNECT_MOBILE)
            ref_xyzw = cur_xyzw
            delta_xyzw = np.subtract(tar_xyzw, ref_xyzw)
            tar_xyzw = delta_xyzw + ref_xyzw
            cur_xyzw = move_mobile_robot(sock_mobile, cur_xyzw, tar_xyzw, tar_xyzw, MOBILE_IP, CONNECT_MOBILE, move_direct=True)
            if not CONNECT_MOBILE:
                gscene.show_motion(snode_nxt.traj)
            gscene.show_pose(to_state.Q)
    # leave highlight on cleared area
    swp_fin = gscene.copy_from(gscene.NAME_DICT["sweep"], new_name="sweep_fin_{}".format(i_s), color=(0,0,1,0.5))
    swp_fin.dims = (swp_fin.dims[0], swp_fin.dims[1], swp_fin.dims[2]+0.002)
    gscene.update_marker(swp_fin)
    swp_fin_list.append(swp_fin)

curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.12 -0.01  0.37 -0.53]
target xyzw: [-0.14 -0.48  0.37 -0.53]
move to: [-0.14 -0.48  0.37 -0.53]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.19 -0.    0.    1.  ]
target xyzw: [0.31 0.   0.   1.  ]
move to: [0.31 0.   0.   1.  ]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.45  0.35 -0.    1.  ]
target xyzw: [ 0.05  0.35 -0.    1.  ]
move to: [ 0.05  0.35 -0.    1.  ]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
curre  xyzw: [0 0 0 1]
ready  xyzw: [ 1.77  0.48 -0.71  0.71]
target xyzw: [ 1.77 -0.02 -0.71  0.71]
move to: [ 0.59 -0.01  0.    1.  ]
move to: [ 1.18 -0.01  0.    1.  ]
move to: [ 1.77 -0.02 -0.71  0.71]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.55  0.09  0.    1.  ]
target xyzw: [-0.05  0.09  0.    1.  ]
move to: [-0.05  0.09  0.    1.  ]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
curre  xyzw: [0 0 0 1]
ready  xyz

In [13]:
test_fun.clear()
for swp_fin in swp_fin_list:
    gscene.remove(swp_fin)


## 2. Closet cleaning

### 2.2. Make closet cleaning plan

#### 2.2.1 get division-base pose combination data

In [14]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_actor import *
from pkg.planning.constraint.constraint_subject import *

In [29]:
brush_face = pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
                                  point=(-gscene.NAME_DICT['brush_face'].dims[0]/2-CLEARANCE,0,0), 
                                  rpy=(0,np.pi/2*1,0))
ccheck.clear()

In [30]:
div_base_dict, Tsm_keys, surface_div_centers, sqdiv_size, div_num, (ax_step, ax_swp, ax_pln) = \
                        get_division_dict(closet_left, brush_face, robot_config,
                                          plane_val=None, tip_dir="up", TOOL_DIM=TOOL_DIM, 
                                          ccheck=ccheck)

In [32]:
# def get_division_dict(surface, brush_face, robot_config, plane_val, tip_dir, TOOL_DIM, ccheck):
surface, brush_face, robot_config, plane_val, tip_dir, TOOL_DIM, ccheck = closet_left, brush_face, robot_config, None, "up", TOOL_DIM, ccheck
## get data
rtype = robot_config.type.name
sweep_path = os.path.join(SWEEP_DAT_PATH, rtype if tip_dir is None else "{}-{}".format(rtype, tip_dir))
sweep_max = np.loadtxt(sweep_path+"-max.csv", delimiter=",")
sweep_min = np.loadtxt(sweep_path+"-min.csv", delimiter=",")

gcheck = ccheck.gcheck
pscene = gcheck.pscene
gscene = pscene.gscene
crob = pscene.combined_robot

## divide surface with squares
sqdiv_size_ref = np.max(TOOL_DIM)
surf_offset = 0.05
surface_dim_eff = np.subtract(surface.dims[:2],
                              sqdiv_size_ref)  # effective surface dims - except tool dimension to boundaries
div_close = np.ceil(surface_dim_eff[0] / sqdiv_size_ref).astype(np.int)
sqdiv_size = surface_dim_eff[0] / div_close  # reduce square size to fit the closed ends
div_num_ = np.ceil(surface_dim_eff / sqdiv_size).astype(np.int)
surface_dim_eff = np.round(sqdiv_size * div_num_, 5)  # enpand effective size of open ends
div_num = div_num_ + 1
surface_div_centers = [tuple(np.round(-surface_dim_eff / 2 + np.multiply(sqdiv_size, (i, j)), 5))
                       for i, j in product(range(div_num[0]), range(div_num[1]))]

In [40]:
sqdiv_size

0.31333333333333335

In [33]:
surface_div_centers

[(-0.94, 0.0),
 (-0.62667, 0.0),
 (-0.31333, 0.0),
 (0.0, 0.0),
 (0.31333, 0.0),
 (0.62667, 0.0),
 (0.94, 0.0)]

In [34]:
## set axes
ax_pln = 2 if tip_dir is None else 0
ax_swp = 1
if plane_val is not None:
    idx_pln = np.argmin(np.abs(sweep_max[:, ax_pln] - plane_val))
    val_pln = sweep_max[idx_pln, ax_pln]
    idc_pln = np.where(sweep_max[:, ax_pln] == val_pln)[0]
    sweep_max = sweep_max[idc_pln, :]
    sweep_min = sweep_min[idc_pln, :]
else:
    Tbs = surface.get_tf(crob.home_dict)
    robot_base = crob.get_robot_base_dict()[robot_config.get_indexed_name()]
    Tbr = gscene.get_tf(robot_base, crob.home_dict)
    Trs = np.matmul(SE3_inv(Tbr), Tbs)
    div_heights_r = np.matmul(Trs[:3, :2], np.transpose(surface_div_centers)).transpose()[:, 2] + Trs[2, 3]
    idc_h_matchs = []
    for div_rh in div_heights_r:
        idx_min = np.argmin(np.abs(sweep_min[:, 2] - div_rh))
        val_min = sweep_min[idx_min, 2]
        idc_min = np.where(sweep_min[:, 2] == val_min)[0]
        idc_h_matchs += list(idc_min)
    idc_h_matchs = sorted(set(idc_h_matchs))
    sweep_min = sweep_min[idc_h_matchs]
    sweep_max = sweep_max[idc_h_matchs]
ax_step = [ax for ax in [0, 1, 2] if ax not in [ax_pln, ax_swp]][0]  # step axis = not plane axis nor sweep axis

In [36]:
ax_pln, ax_swp, ax_step

(0, 1, 2)

In [38]:
## get all sweep points
swp_points_all = []
for step_val, pln_val, min_val, max_val in zip(sweep_min[:, ax_step], sweep_min[:, ax_pln], sweep_min[:, ax_swp],
                                               sweep_max[:, ax_swp]):
    diff_val = max_val - min_val
    sweep_num = np.floor(diff_val / sqdiv_size).astype(np.int)
    min_val_clip = (max_val + min_val) / 2 - (sweep_num * sqdiv_size / 2) + (sqdiv_size / 2)

    swp_points = np.zeros((sweep_num, 3))
    swp_points[:, ax_swp] = min_val_clip + np.arange(sweep_num) * sqdiv_size
    swp_points[:, ax_step] = step_val
    swp_points[:, ax_pln] = plane_val if plane_val is not None else pln_val
    swp_points_all.append(np.round(swp_points, 3))
swp_points_all = np.concatenate(swp_points_all)

In [39]:
swp_points_all

array([[-7.40e-01, -0.00e+00,  3.40e-01],
       [-7.20e-01, -0.00e+00,  3.40e-01],
       [-7.00e-01, -0.00e+00,  3.40e-01],
       [-6.80e-01, -1.57e-01,  3.40e-01],
       [-6.80e-01,  1.57e-01,  3.40e-01],
       [-6.60e-01, -1.57e-01,  3.40e-01],
       [-6.60e-01,  1.57e-01,  3.40e-01],
       [-6.40e-01, -1.57e-01,  3.40e-01],
       [-6.40e-01,  1.57e-01,  3.40e-01],
       [-6.20e-01, -1.57e-01,  3.40e-01],
       [-6.20e-01,  1.57e-01,  3.40e-01],
       [-6.00e-01, -3.13e-01,  3.40e-01],
       [-6.00e-01,  0.00e+00,  3.40e-01],
       [-6.00e-01,  3.13e-01,  3.40e-01],
       [-5.80e-01, -3.13e-01,  3.40e-01],
       [-5.80e-01, -0.00e+00,  3.40e-01],
       [-5.80e-01,  3.13e-01,  3.40e-01],
       [-5.60e-01, -3.13e-01,  3.40e-01],
       [-5.60e-01, -0.00e+00,  3.40e-01],
       [-5.60e-01,  3.13e-01,  3.40e-01],
       [-5.40e-01, -3.13e-01,  3.40e-01],
       [-5.40e-01, -0.00e+00,  3.40e-01],
       [-5.40e-01,  3.13e-01,  3.40e-01],
       [-5.20e-01, -3.13e-01,  3.4

In [48]:


## make feasible base-div dictionary
ROBOT_BASE = pscene.robot_chain_dict[robot_config.get_indexed_name()]["link_names"][0]
TIP_LINK = brush_face.geometry.link_name
Tbs = surface.get_tf(crob.home_dict)
Tmr = gcheck.gscene.get_tf(to_link=ROBOT_BASE, from_link=robot_config.root_on, Q=crob.home_dict)
Trm = SE3_inv(Tmr)
if tip_dir is None:
    Rre = Rot_rpy([np.pi, np.pi/2, 0])
elif tip_dir == "up":
    Rre = Rot_rpy([0, 0, np.pi])
elif tip_dir == "down":
    Rre = Rot_rpy([np.pi, 0, 0])
else:
    raise(RuntimeError("Not defined"))

In [44]:
np.round(Rot_rpy([np.pi, np.pi/2, 0]))

array([[ 0.,  0., -1.],
       [ 0., -1., -0.],
       [-1.,  0., -0.]])

In [47]:
Rot2rpy(np.round(np.matmul(Rot_axis(2, -np.pi/2), Rot_rpy([np.pi, np.pi/2, 0]))))

array([ 3.14159265, -0.        ,  0.        ])

In [42]:
Rre

array([[ 1.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00, -1.0000000e+00, -1.2246468e-16],
       [ 0.0000000e+00,  1.2246468e-16, -1.0000000e+00]])

In [49]:

div_base_dict = defaultdict(lambda: defaultdict(list))
Tbm_in_all = []
Tbm_fail_all = []
Tbm_succ_all = []
for i_div, div_center in enumerate(surface_div_centers):
        Tsc = SE3(np.identity(3), tuple(div_center)+(surface.dims[2]/2,))
        for i in range(4) if tip_dir is None else [0,2]:
            Tct = SE3(Rot_axis(3, i * np.pi/2), [0, 0, 0])
            Tst = matmul_series(Tsc, Tct)
            for swp_point in swp_points_all:
                Tre = SE3(Rre, swp_point)
                Tet = brush_face.get_tf_handle(crob.home_dict, from_link=TIP_LINK)
                Trt = matmul_series(Tre, Tet)
                Tsr = matmul_series(Tst, SE3_inv(Trt))
                Tsm = np.matmul(Tsr, Trm)
                Tbm = np.matmul(Tbs, Tsm)
                if (np.all(np.abs(Tsm[:2,3]) < np.divide(surface.dims[:2], 2))
                    or (np.abs(Tsm[0,3]) > np.divide(surface.dims[0], 2))): # mobile loc inside surface
                    Tbm_in_all.append(Tbm)
                    continue
                if ccheck(T_loal=Tbm, Q_dict=crob.home_dict): #check feasible
                    Tbm_succ_all.append(Tbm)
                    Tsm_xq = T2xyzquat(Tsm)
                    Tsm_key = tuple(np.round(Tsm_xq[0], 3)), tuple(np.round(Tsm_xq[1], 3))
                    div_base_dict[Tsm_key][i].append(i_div)
                else:
                    Tbm_fail_all.append(Tbm)
Tsm_keys = sorted(div_base_dict.keys())
# return div_base_dict, Tsm_keys, surface_div_centers, sqdiv_size, div_num, (ax_step, ax_swp, ax_pln)

In [50]:
Tsm_keys

[((-0.913, -0.372, 1.079), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.331, 1.179), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.315, 1.118), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.314, 1.099), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.313, 1.059), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.311, 1.159), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.307, 1.138), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.267, 1.039), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.26, 1.019), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.245, 0.998), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.228, 0.979), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.213, 0.958), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.21, 0.938), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.195, 0.919), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.175, 0.878), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.174, 0.899), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.168, 1.259), (-0.0, 0.707, -0.0, 0.707)),
 ((-0.913, -0.165, 0.859), (-0.0, 0.707, -0.0, 0.7

In [31]:
Tsm_keys

[((-0.978, -0.506, 0.075), (0.707, 0.0, 0.707, 0.0)),
 ((-0.665, -0.506, 0.075), (0.707, 0.0, 0.707, 0.0)),
 ((-0.351, -0.506, 0.075), (0.707, 0.0, 0.707, 0.0)),
 ((-0.038, -0.506, 0.075), (0.707, 0.0, 0.707, 0.0))]

#### 2.2.2 select base poses and generate motions

In [17]:
HOME_POSE_SWEEP = VIEW_POSE_EXT
crob.home_pose = HOME_POSE_SWEEP
crob.home_dict = list2dict(crob.home_pose, gscene.joint_names)
floor_ws = gscene.NAME_DICT["floor_ws"]    
test_fun_cl = TestBaseDivFunc(ppline, floor_ws, closet_left, sqdiv_size, WP_DIMS, TOOL_DIM, crob.home_dict)

In [27]:
test_fun_cl.clear()

idx_bases, idc_divs, covered_all, test_output_dict = select_max_cover_bases(
    div_base_dict, Tsm_keys, surface_div_centers, div_num, ax_step, 
    test_fun=test_fun_cl)

Use 36/36 agents
======================= terminated 14: max iteration time reached (1633464549/1633464546.74 s) ===============================
======================= terminated 33: max iteration time reached (1633464549/1633464546.74 s) ===============================
======================= terminated 17: max iteration time reached (1633464549/1633464546.74 s) ===============================
======================= terminated 35: max iteration time reached (1633464549/1633464546.74 s) ===============================
======================= terminated 22: max iteration time reached (1633464549/1633464546.74 s) ===============================
======================= terminated 16: max iteration time reached (1633464549/1633464546.74 s) ===============================
======================= terminated 13: max iteration time reached (1633464549/1633464546.74 s) ===============================
======================= terminated 21: max iteration time reached (1633464549/1633464546.74 s)

======================= terminated 7: max iteration time reached (1633464550/1633464546.74 s) ===============================
======================= terminated 9: max iteration time reached (1633464550/1633464546.74 s) ===============================
======================= terminated 31: max iteration time reached (1633464550/1633464546.74 s) ===============================
======================= terminated 32: max iteration time reached (1633464550/1633464546.74 s) ===============================
======================= terminated 3: max iteration time reached (1633464550/1633464546.74 s) ===============================
======================= terminated 15: max iteration time reached (1633464550/1633464546.74 s) ===============================
======================= terminated 12: max iteration time reached (1633464550/1633464546.74 s) ===============================
======================= terminated 8: max iteration time reached (1633464550/1633464546.74 s) ====================

======================= terminated 20: max iteration time reached (1633464554/1633464551.71 s) ===============================
======================= terminated 15: max iteration time reached (1633464554/1633464551.71 s) ===============================
======================= terminated 24: max iteration time reached (1633464554/1633464551.71 s) ===============================
======================= terminated 19: max iteration time reached (1633464554/1633464551.71 s) ===============================
======================= terminated 25: max iteration time reached (1633464554/1633464551.71 s) ===============================
======================= terminated 29: max iteration time reached (1633464554/1633464551.71 s) ===============================
======================= terminated 14: max iteration time reached (1633464554/1633464551.71 s) ===============================
======================= terminated 27: max iteration time reached (1633464554/1633464551.71 s) ================

======================= terminated 17: max iteration time reached (1633464560/1633464557.22 s) ===============================
======================= terminated 16: max iteration time reached (1633464560/1633464557.22 s) ===============================
======================= terminated 21: max iteration time reached (1633464560/1633464557.22 s) ===============================
======================= terminated 20: max iteration time reached (1633464560/1633464557.22 s) ===============================
======================= terminated 27: max iteration time reached (1633464560/1633464557.22 s) ===============================
======================= terminated 25: max iteration time reached (1633464560/1633464557.22 s) ===============================
======================= terminated 24: max iteration time reached (1633464560/1633464557.22 s) ===============================
======================= terminated 23: max iteration time reached (1633464560/1633464557.22 s) ================

======================= terminated 11: max iteration time reached (1633464560/1633464557.22 s) ===============================
======================= terminated 33: max iteration time reached (1633464560/1633464557.22 s) ===============================
======================= terminated 7: max iteration time reached (1633464560/1633464557.22 s) ===============================
======================= terminated 2: max iteration time reached (1633464560/1633464557.22 s) ===============================
======================= terminated 13: max iteration time reached (1633464560/1633464557.22 s) ===============================
======================= terminated 14: max iteration time reached (1633464560/1633464557.22 s) ===============================
======================= terminated 30: max iteration time reached (1633464560/1633464557.22 s) ===============================
======================= terminated 19: max iteration time reached (1633464560/1633464557.22 s) ==================

======================= terminated 30: max iteration time reached (1633464563/1633464560.69 s) ===============================
======================= terminated 5: max iteration time reached (1633464563/1633464560.69 s) ===============================
======================= terminated 25: max iteration time reached (1633464563/1633464560.69 s) ===============================
======================= terminated 2: max iteration time reached (1633464563/1633464560.69 s) ===============================
======================= terminated 1: max iteration time reached (1633464563/1633464560.69 s) ===============================
======================= terminated 6: max iteration time reached (1633464563/1633464560.69 s) ===============================
======================= terminated 13: max iteration time reached (1633464563/1633464560.69 s) ===============================
======================= terminated 32: max iteration time reached (1633464563/1633464560.69 s) ====================

#### 2.2.3 refine motions

In [10]:
HOME_POSE_MOVE = VIEW_POSE
snode_schedule_list, scene_args_list, scene_kwargs_list = refine_order_plan(
    ppline, snode_schedule_dict, idx_bases, idc_divs, Qcur, 
    floor_ws, wayframer, bed_mat, Tsm_keys, surface_div_centers, sqdiv_size, 
    WP_DIMS, TOOL_DIM, ROBOT_NAME, MOBILE_NAME, HOME_POSE_MOVE)

Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached


### 2.3 Execute bed cleaning sequence

In [11]:
# indy.traj_vel = 3

In [12]:
swp_fin_list = []
for i_s, (snode_schedule, sargs, skwargs) in enumerate(zip(snode_schedule_list, scene_args_list, scene_kwargs_list)):
    set_base_sweep(*sargs, **skwargs)
    for snode_pre, snode_nxt in zip(snode_schedule[:-1], snode_schedule[1:]):
        from_state = snode_pre.state
        to_state = snode_nxt.state
        subjects, ok = pscene.get_changing_subjects(from_state, to_state)

        if len(subjects) ==0 or subjects[0] == "sweep":
            if CONNECT_INDY:
                ppline.execute_schedule([snode_pre, snode_nxt], one_by_one=True)
                with indy:
                    time.sleep(0.5)
                    indy.wait_for_move_finish()
            else:
                ppline.play_schedule([snode_pre, snode_nxt])

        elif subjects[0] == "waypoints":
            Tbw0 = wayframer.get_tf_handle(list2dict(from_state.Q, gscene.joint_names))
            Tbw1 = wayframer.get_tf_handle(list2dict(to_state.Q, gscene.joint_names))
            Tw0w1 = np.matmul(SE3_inv(Tbw0), Tbw1)
            cur_xyzw, tar_xyzw_rd, tar_xyzw = get_relative_mobile_command(Tw0w1, CONNECT_MOBILE)
            ref_xyzw = cur_xyzw
            delta_xyzw = np.subtract(tar_xyzw, ref_xyzw)
            tar_xyzw = delta_xyzw + ref_xyzw
            cur_xyzw = move_mobile_robot(sock_mobile, cur_xyzw, tar_xyzw, tar_xyzw, MOBILE_IP, CONNECT_MOBILE, move_direct=True)
            if not CONNECT_MOBILE:
                gscene.show_motion(snode_nxt.traj)
            gscene.show_pose(to_state.Q)
    # leave highlight on cleared area
    swp_fin = gscene.copy_from(gscene.NAME_DICT["sweep"], new_name="sweep_fin_{}".format(i_s), color=(0,0,1,0.5))
    swp_fin.dims = (swp_fin.dims[0], swp_fin.dims[1], swp_fin.dims[2]+0.002)
    gscene.update_marker(swp_fin)
    swp_fin_list.append(swp_fin)

curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.12 -0.01  0.37 -0.53]
target xyzw: [-0.14 -0.48  0.37 -0.53]
move to: [-0.14 -0.48  0.37 -0.53]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.19 -0.    0.    1.  ]
target xyzw: [ 0.31 -0.    0.    1.  ]
move to: [ 0.31 -0.    0.    1.  ]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.45  0.35 -0.    1.  ]
target xyzw: [ 0.05  0.35 -0.    1.  ]
move to: [ 0.05  0.35 -0.    1.  ]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
curre  xyzw: [0 0 0 1]
ready  xyzw: [ 1.77  0.48  0.71 -0.71]
target xyzw: [ 1.77 -0.02  0.71 -0.71]
move to: [ 0.59 -0.01  0.    1.  ]
move to: [ 1.18 -0.02  0.    1.  ]
move to: [ 1.77 -0.02  0.71 -0.71]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.55  0.09  0.    1.  ]
target xyzw: [-0.05  0.09  0.    1.  ]
move to: [-0.05  0.09  0.    1.  ]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
curre  xyzw: [0 0 0 1]
re

## 3. Return to home